In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
# from keras.layers.core import Dense,Activation,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [3]:
hotel_details=pd.read_csv(r'DataSet\Hotel_details.csv')
hotel_rooms=pd.read_csv(r'DataSet\Hotel_Room_attributes.csv')
hotel_cost=pd.read_csv(r'DataSet\hotels_RoomPrice.csv')

In [4]:
hotel_details.head()


,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR


In [5]:
hotel_rooms.head()

,id,hotelcode,roomamenities,roomtype,ratedescription
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed"
1,50672149,8328096,Air conditioning: ;Closet: ;Fireplace: ;Free W...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
2,50643430,8323442,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
3,50650317,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
4,50650318,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"


Data Cleaning and transformations

In [6]:
del hotel_details['id']
del hotel_rooms['id']
del hotel_details['zipcode']

In [7]:
hotel_details=hotel_details.dropna()
hotel_rooms=hotel_rooms.dropna()


In [8]:
hotel_details.drop_duplicates(subset='hotelid',keep=False,inplace=True)

In [9]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotelcode',right_on='hotelid',how='inner')

In [10]:
hotel['hotelcode'].isna().count()


145425

In [11]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription', 'hotelid',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'Source', 'url', 'curr'],
      dtype='object')

In [12]:
hotel['roomtype']

0                            Double Room
1                            Double Room
2         Deluxe Double Room with Shower
3                   Superior Double Room
4           Standard Double or Twin Room
                       ...              
145420                Standard Twin Room
145421                Standard Twin Room
145422       1 Double Bed, standard room
145423       1 Double Bed, standard room
145424      2 Single Beds, standard room
Name: roomtype, Length: 145425, dtype: object

Recommender system based only on City and ratings about the hotel

In [17]:
def citybased(city):
    hotel['city']=hotel['city'].str.lower()
    citybase=hotel[hotel['city']==city.lower()]
    citybase=citybase.sort_values(by='starrating',ascending=False)
    citybase.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if(citybase.empty==0):
        hname=citybase[['hotelname','starrating','address','roomamenities','ratedescription']]
        return hname.head()
    else:
        print('No Hotels Available')

In [14]:
hotel['city']

0         Nether Stowey
1         Nether Stowey
2         Nether Stowey
3         Nether Stowey
4         Nether Stowey
              ...      
145420         Merignac
145421         Merignac
145422         Merignac
145423         Merignac
145424         Merignac
Name: city, Length: 145425, dtype: object

In [18]:
print('Top 5 hotels')
citybased('London')

Top 5 hotels


,hotelname,starrating,address,roomamenities,ratedescription
48985,The Colonnade London Hotel,4,2 Warrington Crescent,Additional toilet: ;Air conditioning: ;Alarm c...,"Room size: 28 m²/301 ft², Balcony/terrace, Non..."
58663,South Point Suites - London Bridge,4,Bermondsey Street,Air conditioning: ;Alarm clock: ;Blackout curt...,"Room size: 24 m²/258 ft², Street view, Non-smo..."
106943,Doubletree by Hilton London Chelsea,4,Imperial Road Imperial Wharf,Air conditioning: ;Alarm clock: ;Blackout curt...,"Room size: 20 m²/215 ft², Street view, Non-smo..."
57452,Sir Christopher Wren Hotel & Spa,4,Unknown,Air conditioning: ;Alarm clock: ;Bathrobes: ;B...,"Room size: 25 m²/269 ft², Non-smoking, Shower ..."
92479,The Manor Hotel Heathrow,4,"Village Green, Datchet,",Additional bathroom: ;Additional toilet: ;Air ...,"Shower and bathtub, 1 double bed"
58886,Holiday Inn Express London - Epsom Downs,4,Langley Vale Road,Air conditioning: ;Carpeting: ;Closet: ;Clothe...,"Room size: 25 m²/269 ft², Non-smoking, Shower,..."
57026,Ten Manchester Street Hotel,4,10 Manchester Street,Air conditioning: ;Bathrobes: ;Blackout curtai...,"Room size: 16 m²/172 ft², Non-smoking, Shower,..."
55913,Acorn Lodge Gatwick,4,"79 Massetts Road, Horley",Air conditioning: ;Carpeting: ;Closet: ;Coffee...,"Shower, 1 double bed and 1 bunk bed"
52224,Moxy London Heathrow Airport,4,804-834 Bath Road,Air conditioning: ;Carpeting: ;Clothes rack: ;...,"Room size: 17 m²/183 ft², Non-smoking, Shower,..."
60709,Hallmark Hotel London Chigwell Prince Regent,4,Manor Road,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,"Pay nothing until January 4, 2020, Free Wi-Fi,..."


In [16]:
citybased('Borovets')


,hotelname,starrating,address,roomamenities,ratedescription
144631,Hotel Krusharskata Kashta,3,"Bul.Iskar 111, s. Govedartsi",Air conditioning: ;Closet: ;Desk: ;Free Wi-Fi ...,"Room size: 22 m²/237 ft², Balcony/terrace, Sho..."
